<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/ashrae_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd /root

/root


In [0]:
!mkdir .kaggle 

In [0]:
!cp '/gdrive/My Drive/Colab_stuff/kaggle.json' '/root/.kaggle/kaggle.json' 
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
cd /content/ 

/content


In [0]:
!mkdir ashrae_energy_prediction

In [7]:
cd ashrae_energy_prediction/

/content/ashrae_energy_prediction


In [8]:
!kaggle competitions download -c ashrae-energy-prediction

 94% 112M/120M [00:01<00:00, 93.3MB/s]
100% 120M/120M [00:01<00:00, 112MB/s] 
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 57.8MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 83.6MB/s]
 86% 76.0M/88.4M [00:00<00:00, 105MB/s] 
100% 88.4M/88.4M [00:00<00:00, 147MB/s]
 92% 154M/167M [00:01<00:00, 132MB/s]
100% 167M/167M [00:01<00:00, 138MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 156MB/s]


In [9]:
!ls

building_metadata.csv	   test.csv.zip   weather_test.csv.zip
sample_submission.csv.zip  train.csv.zip  weather_train.csv.zip


In [10]:
!unzip train.csv.zip
!unzip test.csv.zip
!unzip weather_train.csv.zip
!unzip weather_test.csv.zip
!unzip sample_submission.csv.zip


Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  weather_train.csv.zip
  inflating: weather_train.csv       
Archive:  weather_test.csv.zip
  inflating: weather_test.csv        
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [11]:
!ls

building_metadata.csv	   test.csv.zip      weather_test.csv.zip
sample_submission.csv	   train.csv	     weather_train.csv
sample_submission.csv.zip  train.csv.zip     weather_train.csv.zip
test.csv		   weather_test.csv


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [15]:
df_train = pd.read_csv('train.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_train.info()
df_train.head()
df_train.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


,building_id,meter,timestamp,meter_reading
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575
20216099,1448,0,2016-12-31 23:00:00,2.850


In [16]:
df_test = pd.read_csv('test.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_test.info()
df_test.head()
df_test.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.2 GB


,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


,row_id,building_id,meter,timestamp
41697595,41697595,1444,0,2018-05-09 07:00:00
41697596,41697596,1445,0,2018-05-09 07:00:00
41697597,41697597,1446,0,2018-05-09 07:00:00
41697598,41697598,1447,0,2018-05-09 07:00:00
41697599,41697599,1448,0,2018-05-09 07:00:00


In [17]:
df_train['train_test'] = 0
df_test['train_test'] = 1
df_test.drop(columns = ['row_id'],inplace=True)

,building_id,meter,timestamp,train_test
0,0,0,2017-01-01 00:00:00,1
1,1,0,2017-01-01 00:00:00,1
2,2,0,2017-01-01 00:00:00,1
3,3,0,2017-01-01 00:00:00,1
4,4,0,2017-01-01 00:00:00,1
...,...,...,...,...
41697595,1444,0,2018-05-09 07:00:00,1
41697596,1445,0,2018-05-09 07:00:00,1
41697597,1446,0,2018-05-09 07:00:00,1
41697598,1447,0,2018-05-09 07:00:00,1


In [18]:
df_full = pd.concat([df_train, df_test],axis=0,ignore_index=True,sort=False)
df_full.info()
df_full.head()
df_full.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61913700 entries, 0 to 61913699
Data columns (total 6 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
train_test       int64
row_id           float64
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 2.8 GB


,building_id,meter,timestamp,meter_reading,train_test,row_id
0,0,0,2016-01-01,0.0,0,NaN
1,1,0,2016-01-01,0.0,0,NaN
2,2,0,2016-01-01,0.0,0,NaN
3,3,0,2016-01-01,0.0,0,NaN
4,4,0,2016-01-01,0.0,0,NaN


,building_id,meter,timestamp,meter_reading,train_test,row_id
61913695,1444,0,2018-05-09 07:00:00,NaN,1,41697595.0
61913696,1445,0,2018-05-09 07:00:00,NaN,1,41697596.0
61913697,1446,0,2018-05-09 07:00:00,NaN,1,41697597.0
61913698,1447,0,2018-05-09 07:00:00,NaN,1,41697598.0
61913699,1448,0,2018-05-09 07:00:00,NaN,1,41697599.0


In [0]:
df_full.drop(columns='row_id',inplace=True)

In [0]:
sns.pointplot(data=df_full, x='timestamp',y='meter_reading',hue='train_test')